In [ ]:
import numpy as np
import pydicom
import os
import matplotlib.pyplot as plt
# from glob import glob
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
# from sklearn.cluster import KMeans
from skimage import filters

import tensorflow as tf
# from tensorflow.keras import Model, Sequential 
# from tensorflow.keras.utils import Sequence
# from tensorflow.keras.layers import Conv2D, MultiHeadAttention, AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D , Conv3D, Layer, MaxPooling2D, Dropout, Flatten, Dense, GRU, ConvLSTM2D, Input, BatchNormalization, TimeDistributed, MaxPooling3D, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from random import shuffle
# from tensorflow.keras.regularizers import l2

# from tensorflow.keras import layers
from tensorflow import keras
# from keras.models import Model, load_model

import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)
print(keras.__version__)
import pydicom
import re
import math
import random
from PIL import Image

np.random.seed(1234)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import warnings
warnings.filterwarnings(action='once')

MAX_SEQ_LENGTH = 200
IMG_SIZE = 512
image_path = "/home/data/pe/test"
NUM_SCANS = 8
NUM_CHANNELS = 1
INPUT_DIM = 256
NUM_FEATURES = num_features = 512
BATCH_SIZE = 1

LAYER_BACK = 4
import cv2
import gc
import datetime

In [ ]:
all_ids = pd.read_csv('/home/data/pe/train.csv')
all_ids_test = pd.read_csv('/home/data/pe/test.csv')

In [ ]:
import segmentation_models as sm
from segmentation_models import Unet
from keras.layers import Input, Conv2D
from keras.models import Model

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

segment_model = Unet(backbone_name='resnet34', encoder_weights=None, input_shape=(None, None, 1))

segment_model.trainable = False

segment_model.load_weights('/home/shared/model_checkpoint_paige/segment/segment_lungs')

segment_model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)


In [ ]:
def get_slice_data(slice, verbose = False, dim=256, folder='nps/imgs'):
    study_id = slice.StudyInstanceUID
    series_id = slice.SeriesInstanceUID
    SOP_id = slice.SOPInstanceUID
    
    try:
        data = pydicom.dcmread(os.path.join(image_path, study_id, series_id, SOP_id + '.dcm'))
    except:
        return 'ERROR'
    
    ycoord = data.get_item([0x0020, 0x0032])
    ycoord = float(str(ycoord.value).split('\\')[-1].replace("'",''))
    intercept = int(float(data.get_item([0x0028, 0x1052]).value)) 
    slope = int(float(data.get_item([0x0028, 0x1053]).value))                
    frame = np.asarray(data.pixel_array  * slope + intercept)
    
    if verbose:
        fig, ax = plt.subplots(2, 2, figsize=[24, 24])
        ax[0][0].imshow(frame)
        ax[0][0].axis('off')


    frame = frame.reshape((512,512,1)) 
    original = np.copy(frame)
    frame = frame[None, ...]
    
    frame = tf.convert_to_tensor(frame)
    
    frame = preprocess_input(frame)
    mask = np.round(segment_model(frame) + 0.1)
    
    masked = mask*frame
    masked = masked[0]
    
    frame = frame[0]
    mask = mask[0]

    if np.round(mask).sum() > 10000:

        if verbose:
            ax[0][1].imshow(mask)
            ax[0][1].axis('off')


        x_min = np.min(tf.where(mask != 0)[:,0])
        x_max = np.max(tf.where(mask != 0)[:,0])
        y_min = np.min(tf.where(mask != 0)[:,1])
        y_max = np.max(tf.where(mask != 0)[:,1])

        frame = np.array(frame[x_min:x_max,y_min:y_max]).astype('float32')
        masked = np.array(masked[x_min:x_max,y_min:y_max]).astype('float32')
        original = np.array(original).astype('float32')

        img = np.zeros([dim,dim,3])
        img[:,:,0] = cv2.resize(frame,(dim,dim))
        img[:,:,1] = cv2.resize(masked,(dim,dim))
        img[:,:,2] = cv2.resize(original,(dim,dim))

        if verbose:
            ax[1][0].imshow(frame)
            ax[1][0].axis('off')

        img = (np.clip(img,-250, 450)-100)/350

        if verbose:
            print(np.max(frame), np.min(frame))
            print(np.max(img), np.min(img))
            ax[1][1].imshow(img[:,:,0])
            ax[1][1].axis('off')
            plt.show()

        id_str = study_id+'_'+series_id+'_'+SOP_id + '.png'
        img = (img +1)*(255/2)

        cv2.imwrite('/home/shared/' + folder+ + id_str, img)

    return ycoord

In [ ]:
all_ids = all_ids.sort_values(by=['StudyInstanceUID','SOPInstanceUID']).reset_index(drop=True)
all_ids_test = all_ids_test.sort_values(by=['StudyInstanceUID','SOPInstanceUID']).reset_index(drop=True)

In [ ]:
all_ids['ycoord'] = all_ids.apply(lambda x: get_slice_data(x), axis=1)
all_ids_test['ycoord'] = all_ids_test.apply(lambda x: get_slice_data(x), axis=1, folder='test/imgs/')

In [ ]:
directory = '/home/shared/nps/imgs/'

lisdir = os.listdir(directory)http://172.27.10.122:6831/user/paige/notebooks/all%20data%20generate.ipynb#

files = pd.DataFrame({'file_name':lisdir})
files = pd.DataFrame(data=files['file_name'].str.replace('.png','').str.split('_').to_list(), columns=['StudyInstanceUID', 'SeriesInstanceUID','SOPInstanceUID'])
files['contains_lung'] = True

all_ids = pd.merge(all_ids, files, how='outer')
all_ids = all_ids.fillna(False)

In [ ]:
directory = '/home/shared/test/imgs/'

lisdir = os.listdir(directory)

files = pd.DataFrame({'file_name':lisdir})
files = pd.DataFrame(data=files['file_name'].str.replace('.png','').str.split('_').to_list(), columns=['StudyInstanceUID', 'SeriesInstanceUID','SOPInstanceUID'])
files['contains_lung'] = True

all_ids_test = pd.merge(all_ids_test, files, how='outer')
all_ids_test = all_ids_test.fillna(False)

In [ ]:
all_ids.to_csv('all_ids_updated.csv')
all_ids_test.to_csv('test_ids.csv')

In [ ]:
all_studies = all_ids.drop_duplicates('StudyInstanceUID')
all_studies = all_studies.sample(frac=1).reset_index(drop=True)
all_studies['fold'] = all_studies.index % 10
all_studies = all_studies[['StudyInstanceUID','fold']]

In [ ]:
# all_studies.to_csv('fold_df.csv')